# GitHub Tracking Application Part 1

In [1]:
from pixiedust.display.app import *
import requests
import pandas

@PixieApp
class GitHubTracking():
    @route()
    def main_screen(self):
        return """
<style>
    div.outer-wrapper {
        display: table;width:100%;height:300px;
    }
    div.inner-wrapper {
        display: table-cell;vertical-align: middle;height: 100%;width: 100%;
    }
</style>
<div class="outer-wrapper">
    <div class="inner-wrapper">
        <div class="col-sm-3"></div>
        <div class="input-group col-sm-6">
          <input id="query{{prefix}}" type="text" class="form-control" placeholder="Search projects on GitHub">
          <span class="input-group-btn">
            <button class="btn btn-default" type="button" pd_options="query=$val(query{{prefix}})">Submit Query</button>
          </span>
        </div>
    </div>
</div>   
"""
    @route(query="*")
    @templateArgs
    def do_search(self, query):
        self.first_url = "https://api.github.com/search/repositories?q={}".format(query)
        self.prev_url = None
        self.next_url = None
        self.last_url = None
        
        response = requests.get(self.first_url)
        if not response.ok:
            return "<div>An Error occurred: {{response.text}}</div>"

        total_count = response.json()['total_count']
        self.next_url = response.links.get('next', {}).get('url', None)
        self.last_url = response.links.get('last', {}).get('url', None)
        return """
<h1><center>{{total_count}} repositories were found</center></h1>
<ul class="pagination">
  <li><a href="#" pd_options="page=first_url" pd_target="body{{prefix}}">First</a></li>
  <li><a href="#" pd_options="page=prev_url" pd_target="body{{prefix}}">Prev</a></li>
  <li><a href="#" pd_options="page=next_url" pd_target="body{{prefix}}">Next</a></li>
  <li><a href="#" pd_options="page=last_url" pd_target="body{{prefix}}">Last</a></li>
</ul>
<table class="table">
    <thead>
        <tr>
            <th>Repo Name</th>
            <th>Lastname</th>
            <th>URL</th>
            <th>Stars</th>
        </tr>
    </thead>
    <tbody id="body{{prefix}}">
        {{this.invoke_route(this.do_retrieve_page, page='first_url')}}
    </tbody>
</table>
"""
    @route(page="*")
    @templateArgs
    def do_retrieve_page(self, page):
        url = getattr(self, page)
        if url is None:
            return "<div>No more rows</div>"
        response = requests.get(url)
        self.prev_url = response.links.get('prev', {}).get('url', None)
        self.next_url = response.links.get('next', {}).get('url', None)
        items = response.json()['items']
        return """
{%for row in items%}
<tr>
    <td>{{row['name']}}</td>
    <td>{{row.get('owner',{}).get('login', 'N/A')}}</td>
    <td><a href="{{row['html_url']}}" target="_blank">{{row['html_url']}}</a></td>
    <td>{{row['stargazers_count']}}</td>
</tr>
{%endfor%}
        """

app = GitHubTracking()
app.run()